In [2]:
# Import necessary libraries
from google.colab import drive
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from PIL import Image

# # # Mount Google Drive
drive.mount('/content/drive')

# # Change the directory to the location of the data
os.chdir("/content/drive/My Drive/Poisson_Study/data")

Mounted at /content/drive


In [3]:
import numpy as np
import cv2
import glob
import pandas as pd
from skimage.transform import radon, iradon, rescale
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
from skimage.io import imread
import matplotlib.pyplot as plt
from os.path import basename

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Poisson_Study/labels.csv")

In [5]:
def prepare_images(file_paths, image_size=(256, 256), theta=np.linspace(0., 180., 180, endpoint=False)):
    img_array = []
    labels = []
    for file_path in file_paths:
        img = imread(file_path, as_gray=True)
        if img is None:
            print("Failed to load image:", file_path)
            continue

        img = rescale(img, scale=0.4, mode='reflect', multichannel=False, anti_aliasing=False)
        sinogram = radon(img, theta=theta)
        ## completed -1,1,5,2,2.5 3,4,10,5

        scale_factor = 1.5
        noisy_sinogram = np.random.poisson(sinogram * scale_factor)

        reconstructed_image = iradon(noisy_sinogram, theta=theta, filter_name='hamming')
        reconstructed_image = cv2.resize(reconstructed_image, image_size)

        # Normalize the reconstructed image
        reconstructed_image = np.expand_dims(reconstructed_image, axis=-1) / 255.0
        img_array.append(reconstructed_image)

        # Extract label using filename
        base_name = basename(file_path)
        label_series = df[df['filename'] == base_name]['hemorrhage']
        if not label_series.empty:
            labels.append(label_series.iloc[0])
        else:
            print(f"No label found for {base_name}; skipping image.")

    return np.array(img_array), np.array(labels)


In [6]:
file_paths = glob.glob("/content/drive/My Drive/Poisson_Study/data_large/*.png")
X, y = prepare_images(file_paths)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


<ipython-input-5-84757088f9d3>:11: FutureWarning: `multichannel` is a deprecated argument name for `rescale`. It will be removed in version 1.0. Please use `channel_axis` instead.
  img = rescale(img, scale=0.4, mode='reflect', multichannel=False, anti_aliasing=False)
/usr/local/lib/python3.10/dist-packages/skimage/transform/radon_transform.py:75: UserWarning: Radon transform: image must be zero outside the reconstruction circle
  warn('Radon transform: image must be zero outside the '


In [ ]:



import tensorflow as tf

class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        self.conv_layers = [
            tf.keras.layers.Conv2D(filters, kernel_size, activation='relu', padding="same")
            for _ in range(repetitions)
        ]
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size, strides=strides)

    def call(self, inputs):
        x = inputs
        for conv in self.conv_layers:
            x = conv(x)
        return self.max_pool(x)

class MyVGG(tf.keras.Model):
    def __init__(self, num_classes):
        super(MyVGG, self).__init__()
        self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
        self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
        self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
        self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
        self.block_e = Block(filters=512, kernel_size=3, repetitions=3)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(256, activation='relu')
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs, return_last_conv=False):
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)  # Last convolutional output
        if return_last_conv:
            return x
        x = self.flatten(x)
        x = self.fc(x)
        return self.classifier(x)




def call(self, inputs):
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

def prepare_images(file_paths, image_size=(256, 256)):
    img_array = []
    labels = []
    for file_path in file_paths:
        base_name = os.path.basename(file_path)

        # Read and resize the image
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, image_size)
            img = np.expand_dims(img, axis=-1)  # Add channel dimension for grayscale
            img_array.append(img)

            # Find the corresponding label using the filename
            label_series = df[df['filename'] == base_name]['hemorrhage']
            if not label_series.empty:
                labels.append(label_series.iloc[0])
            else:
                print(f"No label found for {base_name}; skipping image.")
                continue  # Skip this image if no label found
        else:
            print("Failed to load image:", file_path)

    return np.array(img_array) / 255.0, np.array(labels)



# # Retrieve image file paths
# file_paths = glob.glob("/content/drive/My Drive/Poisson_Study/data/*.png")

# # Prepare images and labels
# X, y = prepare_images(file_paths)

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)


model = MyVGG(num_classes=2)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test))
model = MyVGG(num_classes=2)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])




 11/100 [==>...........................] - ETA: 2:13:40 - loss: 0.9129 - accuracy: 0.5057

In [ ]:
model.save_weights("/content/drive/MyDrive/Poisson_Study/vgg.h5")
y_pred = model.predict(X_test)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
y_pred_binary = [np.argmax(a) for a in y_pred]

print("Accuracy of model = ", accuracy_score(y_test, y_pred_binary))
print(classification_report(y_test, y_pred_binary, target_names=['0', '1']))

